In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
import keras

from keras.src.legacy.preprocessing.image import ImageDataGenerator
from keras.src import applications
from keras.models import Sequential, load_model
from keras.src.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Flatten, Dense, Dropout
from keras.src.legacy.preprocessing import image

import cv2
import os

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Data Augmentation on train dataset
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Data Augmentation on test dataset
test_datagen = ImageDataGenerator(
    rescale=1./255
)

In [3]:
train_generator = train_datagen.flow_from_directory(
    'train',
    target_size=(255,255),
    batch_size=32,
    class_mode='categorical'
)

valid_generator = test_datagen.flow_from_directory(
    'valid',
    target_size=(255,255),
    batch_size=32,
    class_mode='categorical'
)

Found 13104 images belonging to 15 classes.
Found 300 images belonging to 15 classes.


In [4]:
model = Sequential()
model.add(Conv2D(32,(3,3),input_shape=(255,255,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dense(15,activation='softmax'))

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 253, 253, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 126, 126, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 124, 124, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 246016)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │    15,745,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 15)             │           975 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,765,455 (60.14 MB)

 Trainable params: 15,765,455 (60.14 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
history = model.fit(
        train_generator,
        batch_size=32,
        epochs=20,
        validation_data=valid_generator,
        validation_batch_size=32,
)

Epoch 1/20
  7/410 ━━━━━━━━━━━━━━━━━━━━ 13:13 2s/step - accuracy: 0.0451 - loss: 5.7191

KeyboardInterrupt: 

In [6]:
model=load_model('WheatDiseaseDetection.h5')

In [11]:
test_generator = test_datagen.flow_from_directory(
    'test',
    target_size=(255,255),
    batch_size=32,
    class_mode='categorical'
)

Found 750 images belonging to 15 classes.


In [12]:
model.predict(test_generator)

24/24 ━━━━━━━━━━━━━━━━━━━━ 10s 413ms/step


array([[8.9320338e-01, 6.7748203e-07, 2.2371607e-03, ..., 5.7586806e-04,
        7.7623794e-05, 7.0131860e-09],
       [6.5861408e-05, 9.7585970e-01, 2.0427758e-05, ..., 1.4861709e-07,
        3.4530377e-03, 3.8487784e-08],
       [1.0607783e-01, 8.2231108e-03, 8.1896182e-04, ..., 2.6011938e-04,
        6.6843078e-02, 4.3141916e-03],
       ...,
       [1.9661104e-04, 2.8271007e-04, 1.8477079e-02, ..., 5.8653040e-07,
        1.3864817e-03, 4.7100737e-05],
       [1.1767490e-03, 2.0558273e-06, 1.2964322e-06, ..., 9.9499363e-01,
        1.3789325e-03, 4.7514161e-05],
       [2.4537416e-04, 1.0462538e-04, 1.9642588e-04, ..., 3.7416978e-08,
        2.4973741e-04, 2.3946774e-04]], dtype=float32)

In [15]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples//4)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

187/187 ━━━━━━━━━━━━━━━━━━━━ 9s 47ms/step - accuracy: 0.7607 - loss: 1.5817   
Test Loss: 1.5608
Test Accuracy: 0.7640
